In [1]:
from pinecone import Pinecone as PineconeClient
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda


In [2]:
import os
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = "aws"
PINECONE_INDEX_NAME ="build-a-product"

In [3]:
pinecone = PineconeClient(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

In [4]:
embeddings = OpenAIEmbeddings()
vectorstore = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=embeddings)
retriever = vectorstore.as_retriever()

## RAG Prompt

In [5]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [6]:
chain.invoke("what are minimum viable products?")

'Minimum viable products (MVPs) are a version of a new product that allows a team to collect the maximum amount of validated learning about customers with the least effort. The concept of an MVP is to build something small, typically a fraction of the total envisioned product, that includes just enough features to satisfy early adopters and validate a product idea early in the product development cycle. The goal is to minimize the amount of time and resources spent on developing the product before understanding whether it has a good value proposition and whether it is something that customers find valuable and are willing to use or pay for. This approach helps in testing assumptions, understanding customer needs, and providing insights to iterate and improve the product.'

In [7]:
chain.invoke("what is the product company gap and how can it be solved?")

'The "Product Company Gap" refers to the challenge that companies face when transitioning from having a product with market fit to scaling the business into a larger company. It involves bridging the gap between the initial product development and the broader aspects of building a successful business, such as go-to-market strategies, pricing, and business model development.\n\nTo solve the Product Company Gap, companies need to:\n\n1. Design products with go-to-market fit in mind, ensuring that products are not only well-designed but also easy to sell.\n2. Architect a business model from day one that supports scalability, including thoughtful pricing strategies and a clear value proposition.\n3. Focus on achieving product-market fit with a minimum viable segment, keeping the initial target market small and manageable.\n4. Repeat successful strategies to gradually expand the market reach.\n5. Integrate thinking about the business model and go-to-market strategies from the beginning of t